In [3]:
import sys,re,json
import numpy as np
from pathlib import Path

# import IBL packages 
from atlaselectrophysiology.extract_files import extract_data
from ibllib.atlas import AllenAtlas
atlas = AllenAtlas(25) # always register to the 25 um atlas 

# import PinkRig ephys processng tools 
sys.path.insert(0, r"C:\Users\Experiment\Documents\Github\PinkRigs\Analysis\+kilo\python_")
from ReadSGLXData.readSGLX import readMeta

def ks_to_ibl_format(ephys_path,ks_folder='pyKS',recompute=False):
    # Path to KS output
    if 'pyKS' in ks_folder: 
        ks_folder = r'pyKS\output'
        
    ks_path = ephys_path / ks_folder
    # Save path
    out_path = ks_path / 'ibl_format'
    out_path.mkdir(parents=False, exist_ok=True) # make directory if it does not exist

    # extract the data to ibl_format if it has not been done already.
    if not (out_path / 'cluster_matrics.tsv').is_file() or recompute:
        print('converting data to IBL format ...')
        extract_data(ks_path, ephys_path, out_path) 

    # get the probe tracks if they exist.
    subject_path = ephys_path.parents[3]
    brainreg_path = subject_path / r'histology\registration\brainreg_output\manual_segmentation\standard_space\tracks'
    if brainreg_path.is_dir():
        # get some information from meta file 
        meta = readMeta(list(ephys_path.glob('*.ap.cbin'))[0])

        # serial number
        probe_sn = meta['imDatPrb_sn']

        # shanks that have been recorded. 
        imro = meta['snsShankMap']
        channel_data = re.findall(r"\d+:\d+:\d+:\d+",imro)
        shank_idx = np.array([np.array(re.split(':',chandat)).astype('int')[0] for chandat in channel_data])
        recorded_shanks = np.unique(shank_idx).astype('str')
            # get the xyz_picks if the histology exist.

        for shank in recorded_shanks:
            shank_file_name = 'SN%s_shank%s.npy' % (probe_sn,shank)
            shank_anat_path = brainreg_path / shank_file_name

            if shank_anat_path.is_file():
                # Load in coordinates of track in CCF space (order - apdvml, origin - top, left, front voxel
                xyz_apdvml = np.load(shank_anat_path)
                # Convert to IBL space (order - mlapdv, origin - bregma)
                xyz_mlapdv = atlas.ccf2xyz(xyz_apdvml, ccf_order='apdvml') * 1e6
                xyz_picks = {'xyz_picks': xyz_mlapdv.tolist()}

                # save file the ibl_format output path - the format is following the curernt requirements of the GUI, i.e. 
                # xyz_picks.json if single shank recording
                # else xyz_picks_shank0.json 
                if recorded_shanks.size==1:
                    with open(Path(out_path, 'xyz_picks.json'), "w") as f:
                        json.dump(xyz_picks, f, indent=2)
                else:
                    with open(Path(out_path, 'xyz_picks_shank%s.json' % shank), "w") as f:
                        json.dump(xyz_picks, f, indent=2)
            else: 
                print('the probe/shank IDs do not match.')                
    else: 
        print('the histology does not seem to have been done. ')
        
   

In [ ]:

# Path to raw ephys data
ephys_path = Path(r'Z:\AV014\2022-07-07\ephys\AV014_2022-07-07_NatImagesSparseNoise_g0\AV014_2022-07-07_NatImagesSparseNoise_g0_imec0')
     
ks_to_ibl_format(ephys_path,recompute=False)

converting data to IBL format ...


C:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF:  44%|███████████████████████████▎                                  | 55/125 [00:02<00:02, 26.26it/s]
Extracting waveforms:   0%|                                                              | 0/1671.0704 [00:00<?, ?it/s]
Extracting waveforms:   1%|▌                                                  | 19.0/1671.0704 [00:00<00:31, 51.95it/s]
Extracting waveforms:   6%|██▉                                               | 99.0/1671.0704 [00:00<00:07, 200.74it/s]
Extracting waveforms:  11%|█████▏                                           | 179.0/1671.0704 [00:00<00:05, 257.97it/s]
Extracting waveforms:  16%|███████▉                                         | 269.0/1671.0704 [00:01<00:07, 177.47it/s]
Extracting waveforms:  21%|██████████▏                                    